In [1]:
from openai import OpenAI

In [2]:
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

In [5]:
import os

In [6]:
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [7]:
from datetime import datetime

In [8]:
import requests

In [9]:
from supabase import create_client, Client

In [10]:
supabase_url = os.environ.get("SUPABASE_URL")

In [11]:
supabase_api_key = os.environ.get("SUPBASE_KEY")

In [12]:
supabase: Client = create_client(supabase_url, supabase_api_key)

In [57]:
result = supabase.table('webhooks')\
    .select('id, news_output,source_urls, news_date')\
.gte('news_date', '2026-02-09')\
    .order('created_at', desc=False)\
    .execute()

result.data

[{'id': 305,
  'news_output': 'Amazon announced a plan to spend about $200 billion on capital expenditures, with a large portion aimed at AI-related investments (data centers, chips, robotics, AI capabilities). The commitment was disclosed alongside quarterly results and marks a major increase in AI-related corporate capex.',
  'source_urls': ['https://www.mediapost.com/publications/article/412629/amazon-stuns-observers-with-commitment-to-spend-2.html?edition=141508'],
  'news_date': '2026-02-09'},
 {'id': 306,
  'news_output': "Coverage reports that big tech firms collectively plan roughly $600 billion of AI-related spending in 2026 (aggregating multiple companies' disclosed capex/AI investments), underscoring an industry-wide escalation in infrastructure and AI investment commitments.",
  'source_urls': ['https://m.economictimes.com/tech/artificial-intelligence/big-techs-600-billion-spending-plans-exacerbate-investors-ai-headache/articleshow/128068126.cms'],
  'news_date': '2026-02-0

In [33]:
research_output = []

In [58]:
def openai_research_v2():
    for i in result.data:
        response = client.responses.create(
            model = "gpt-5-nano",
            tools = [{
                "type": "web_search"
            }],
            include = ["web_search_call.action.sources"],
            input = f"""You are an AI research analyst for Krux.

            Your job is to expand a given AI news summary into a structured, fact-checked research brief.

            CRITICAL REQUIREMENTS:
            - You must ONLY include factual information.
            - DO NOT include opinions, interpretations, predictions, or analysis.
            - Every fact MUST be traceable to a credible published source.
            - If a fact cannot be verified from a reliable source, DO NOT include it.
            - Never assume or infer missing information.

            TASK:
            1. Validate the claims made in the summary.
            2. Expand with additional confirmed facts.
            3. Provide supporting citations for every statement.

            OUTPUT FORMAT REQUIRED:
            - Make sure you're just talking about the news and sources
            - Getting straight to the news sources without any background information
            - Make sure the output is in points


            Here are the details of the news that you need to research on:

            {i['news_output']}
            """,
        )

        output = response.output_text
        print(output)

        final_dictionary = {
            'event_id': f"{i['id']}_{i['news_date']}",
            'news_date': i['news_date'],
            'output': output,
            'model_provider': 'openai'
        }

        research_output.append(final_dictionary)
        print(final_dictionary)

        save_research(final_dictionary)

        research_output.append(final_dictionary)

    return research_output

In [59]:
openai_research_v2()

- Claim validated: On February 5, 2026, Amazon announced its Q4 2025 results and stated it expects to invest about $200 billion in capital expenditures across Amazon in 2026. Source: Amazon Q4 2025 press release (Feb 5, 2026) including the forward-looking capex guidance. ([ir.aboutamazon.com](https://ir.aboutamazon.com/news-release/news-release-details/2026/Amazon-com-Announces-Fourth-Quarter-Results/default.aspx))

- Specific focus of the capex plan: The 2026 capex guidance covers areas tied to artificial intelligence and related infrastructure, including AI capabilities, data-center capacity, and hardware investments (chips and robotics), as well as low Earth orbit satellites. Source: Amazon press release notes investments in artificial intelligence and details on Trainium/Graviton chips and other AI-related initiatives; the line announcing “invest about $200 billion in capital expenditures across Amazon in 2026” also appears in the release. ([ir.aboutamazon.com](https://ir.aboutamaz

[{'event_id': '299_2026-02-08',
  'news_date': '2026-02-08',
  'output': '- Verified: The earbuds rumor centers on an internal code name for OpenAI’s first consumer hardware device being Sweetpea. Multiple credible outlets report this codename as part of the leak cycle about the device. ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n\n- Verified: Several credible outlets note that the project is not officially confirmed and remains a leak/rumor at this time. TechCrunch explicitly describes the details as coming from leaks and “tipsters,” not an official OpenAI confirmation; Axios likewise characterizes the information as rumors shared at Davos. ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n\n- Verified: The same reporting that mentions Sweetpea also references a potential second consumer device codenamed Gumdrop in the rumor mill. 

In [50]:
def save_research(research_json):
    supabase.table('research_assistant').insert({
        'event_id': research_json['event_id'],
        'model_provider': research_json['model_provider'],
        'news_date': research_json['news_date'],
        'output': research_json['output']
    }).execute()

In [62]:
a = [{'headline': "Big Tech's $600B AI Spending Spree: Hype or Hardware?", 'output': "Big Tech is going all-in on AI infrastructure in 2026—dropping roughly $600 billion collectively. Amazon, Google, Microsoft, Meta, and Oracle are leading the charge, with Alphabet alone budgeting up to $185 billion. But here's the twist: RBC Capital warns a chunk of that surge is just inflated memory prices (DRAM, HBM), not pure hardware expansion. Meanwhile, Gartner forecasts global AI spending hitting $2.52 trillion this year. This could mean the AI arms race is as much about optics as actual capability—unless ROI finally catches up.", 'sources': [{'name': 'Investing.com', 'url': 'https://www.investing.com/analysis/big-tech-will-spend-600b-on-ai-in-2026-5-stocks-cashing-the-checks-200674615?utm_source=openai'}, {'name': 'Goldman Sachs', 'url': 'https://www.goldmansachs.com/insights/articles/why-ai-companies-may-invest-more-than-500-billion-in-2026?utm_source=openai'}, {'name': 'TrendForce', 'url': 'https://www.trendforce.com/presscenter/news/20251106-12772.html?utm_source=openai'}, {'name': "Barron's", 'url': 'https://www.barrons.com/articles/alphabet-stock-ai-capex-monetization-11f4233c?utm_source=openai'}, {'name': 'Business Insider', 'url': 'https://www.businessinsider.com/big-tech-capex-ai-growth-inflated-memory-prices-rbc-2026-2?utm_source=openai'}, {'name': 'Gartner', 'url': 'https://www.gartner.com/en/newsroom/press-releases/2026-1-15-gartner-says-worldwide-ai-spending-will-total-2-point-5-trillion-dollars-in-2026?utm_source=openai'}], 'model_provider': 'claude', 'news_date': '2026-02-09', 'event_id': '306_2026-02-09'},
{'headline': "Amazon's Dropping $200B on AI in 2026—Wall Street Hates It", 'output': "Amazon just announced a jaw-dropping $200 billion capex plan for 2026—up from roughly $132B in 2025—targeting AI infrastructure, custom chips (Trainium, Graviton), robotics, and satellites. The bombshell dropped alongside solid Q4 results: $213.4B in sales and AWS up 24% to $35.6B. But investors weren't impressed—shares tanked after-hours. This aggressive spend signals Amazon's betting big on the AI arms race, even if Wall Street questions the near-term payoff.", 'sources': [{'name': 'Amazon Q4 2025 Press Release', 'url': 'https://ir.aboutamazon.com/news-release/news-release-details/2026/Amazon-com-Announces-Fourth-Quarter-Results/default.aspx'}, {'name': 'The Information', 'url': 'https://www.theinformation.com/briefings/amazon-plans-200-billion-capital-spending-2026/?utm_source=openai'}, {'name': 'AP News', 'url': 'https://apnews.com/article/f4cfda9dd8ee6e2cdfcfcd90265cf0bb?utm_source=openai'}, {'name': 'TechCrunch', 'url': 'https://techcrunch.com/2026/02/05/amazon-and-google-are-winning-the-ai-capex-race-but-whats-the-prize/?utm_source=openai'}], 'model_provider': 'claude', 'news_date': '2026-02-09', 'event_id': '305_2026-02-09'},
{'headline': "Abu Dhabi's G42 Drops $1B on Vietnam AI Takeover", 'output': "On February 9, 2026, Abu Dhabi's G42 inked a $1 billion deal with Vietnamese partners FPT Corporation and Viet Thai Group to build sovereign AI and cloud infrastructure across Vietnam. The framework backs three data center locations, prioritizing national data sovereignty and digital resilience. Vietnam's aiming to become Southeast Asia's leading AI hub and an AI-native society. This could position Vietnam as the region's digital powerhouse, challenging traditional tech centers like Singapore.", 'sources': [{'name': 'Abu Dhabi Media Office', 'url': 'https://www.mediaoffice.abudhabi/en/technology/g42-signs-us1bn-agreement-with-vietnamese-consortium-to-deploy-national-ai-and-cloud-infrastructure-across-vietnam/'}, {'name': 'Bloomberg', 'url': 'https://www.bloomberg.com/news/articles/2026-02-09/abu-dhabi-s-g42-leads-1-billion-data-center-project-in-vietnam'}, {'name': 'The Business Times', 'url': 'https://www.businesstimes.com.sg/international/asean/ho-chi-minh-city-unveils-twin-us1-billion-digital-asset-ai-data-centre-pacts/'}], 'model_provider': 'claude', 'news_date': '2026-02-09', 'event_id': '308_2026-02-09'},
{'headline': "Australia's Firmus Scores $10B to Build AI Mega-Infrastructure", 'output': "Australian AI infrastructure company Firmus just locked in a massive $10 billion debt package led by Blackstone and Coatue Management to supercharge Project Southgate—its plan to blanket Australia with AI data centres. The funding, done with CDC Data Centres and Nvidia, aims to roll out 1.6 gigawatts of capacity by 2028. Melbourne's already getting 18,500 Nvidia GB300 GPUs online in Q1 2026, while Tasmania's Green AI Campus will host 36,000 GPUs. Firmus previously raised A$830 million in equity, with Nvidia jumping in as an investor. This could cement Australia as a serious AI infrastructure hub outside the US-China axis.", 'sources': [{'name': 'Street Insider', 'url': 'https://www.streetinsider.com/Reuters/Australian%2BAI%2Binfrastructure%2Bdeveloper%2BFirmus%2Blands%2B%2410%2Bbillion%2Bdebt%2Bpackage%2Bfrom%2BBlackstone%2C%2BCoatue/25963549.html'}, {'name': 'Bloomberg', 'url': 'https://www.bloomberg.com/news/articles/2025-10-16/nvidia-partners-in-2-9-billion-australia-data-center-drive?utm_source=openai'}, {'name': 'Firmus', 'url': 'https://firmus.co/newsroom/southgate-expansion?utm_source=openai'}], 'model_provider': 'claude', 'news_date': '2026-02-09', 'event_id': '310_2026-02-09'},
{'headline': "Anthropic's Legal AI Just Tanked Europe's Data Giants", 'output': 'Anthropic dropped Claude Cowork, a plug-in automating contract review, NDA triage, and compliance for legal teams—and Wall Street freaked. Thomson Reuters plunged 18%, Relx 14%, Wolters Kluwer 13%, and Pearson 8% as investors fled traditional legal-data stocks. Days later, Anthropic turbocharged the chaos with Claude Opus 4.6, boosting multi-agent capabilities. Morgan Stanley flagged "heightened competition" in legal data. The selloff spread globally, with traders reassessing whether AI will gut legacy SaaS models. This could mark the beginning of a full-scale disruption for professional-services incumbents.', 'sources': [{'name': 'The Guardian - Anthropic AI Legal Tool', 'url': 'https://www.theguardian.com/technology/2026/feb/03/anthropic-ai-legal-tool-shares-data-services-pearson'}, {'name': 'The Guardian - Software Selloff Live', 'url': 'https://www.theguardian.com/business/live/2026/feb/04/software-stock-selloff-ai-led-disruption-jensen-huang-services-economy-business-live-news-updates?filterKeyEvents=false&page=with%3Ablock-6982f5b88f08cce7778ed53e'}, {'name': 'Investing.com - Anthropic AI Upgrade', 'url': 'https://www.investing.com/news/stock-market-news/anthropic-releases-ai-upgrade-as-market-punishes-software-stocks-4488573?utm_source=openai'}, {'name': 'Investing.com - US Software Stocks Hit', 'url': 'https://www.investing.com/news/stock-market-news/us-software-stocks-hit-by-anthropic-wakeup-call-on-ai-disruption-4485524?utm_source=openai'}], 'model_provider': 'claude', 'news_date': '2026-02-09', 'event_id': '311_2026-02-09'},
{'headline': "DreamTech's Neural4D-2.5 Slashes 3D Content Costs by 95%", 'output': 'DreamTech just dropped Neural4D-2.5, a foundational AI model that turns text and images into production-ready 3D content—at 1 yuan per model. Unveiled in Shenzhen last December and officially announced February 8, it uses a "Native 3D Attribute Grid" architecture with TEXTRIX tech to generate high-fidelity textures, geometry, and watertight models ready for 3D printing. The kicker? Training costs run 1/30th of industry norms, inference at 1/20th, slashing overall production costs by roughly 95%. This could democratize 3D content creation for gaming, manufacturing, and metaverse applications at unprecedented scale.', 'sources': [{'name': 'Newsfile Corp', 'url': 'https://www.newsfilecorp.com/release/282676/DreamTech-Unveils-Neural4D2.5-Introducing-the-Native-3D-Architecture-for-HighFidelity-AIGC'}, {'name': 'Stdaily', 'url': 'https://www.stdaily.com/web/gdxw/2025-12/13/content_446786.html'}, {'name': 'Yesky News', 'url': 'https://news.yesky.com/hotnews/234/325734.shtml'}, {'name': 'China.com Tech', 'url': 'https://tech.china.com/articles/20251209/202512091778497.html'}, {'name': 'Neural4D', 'url': 'https://www.neural4d.com/'}], 'model_provider': 'claude', 'news_date': '2026-02-09', 'event_id': '312_2026-02-09'},
{'headline': 'Ouster Snaps Up StereoLabs to Build Physical AI Powerhouse', 'output': "Lidar leader Ouster (NASDAQ: OUST) closed its $35M+ acquisition of StereoLabs on February 4, 2026, creating what they're calling Physical AI's first unified sensing and perception platform. The deal merges Ouster's digital lidar with StereoLabs' 90,000+ shipped ZED cameras, AI compute, and perception software used by 10,000+ customers across robotics and industrial sectors. StereoLabs pulled in $16M revenue in 2025 and is already EBITDA positive. Co-founders stay on to lead the now-subsidiary. This could accelerate sensor fusion tech for autonomous systems, unifying vision and lidar into one stack.", 'sources': [{'name': 'SEC Form 8-K Filing', 'url': 'https://www.sec.gov/Archives/edgar/data/1816581/000119312526041917/d84917d8k.htm'}, {'name': 'SEC Press Release Exhibit', 'url': 'https://www.sec.gov/Archives/edgar/data/1816581/000119312526041917/d84917dex991.htm'}, {'name': 'Investing.com', 'url': 'https://www.investing.com/news/stock-market-news/ouster-stock-rises-on-acquisition-of-ai-vision-company-stereolabs-93CH-4493896?utm_source=openai'}], 'model_provider': 'claude', 'news_date': '2026-02-09', 'event_id': '313_2026-02-09'},
{'headline': "Ambarella's Bringing Agentic AI to the Physical World", 'output': "Ambarella is hitting Embedded World 2026 (March 10-12, Nuremberg) with live demos of its CV7 Edge AI Vision SoC—capable of processing multiple 8Kp60 video streams on Samsung's efficient 4nm chip. The showcase spans robotics, automotive, industrial automation, and security, all powered by their new DevZone platform that lets developers build edge AI apps with low-code blueprints. This could bridge the gap between chatbots and actual robots that see, think, and act in real-time—making AI genuinely physical.", 'sources': [{'name': 'GlobeNewswire - Embedded World 2026', 'url': 'https://www.globenewswire.com/news-release/2026/02/09/3234319/0/en/Ambarella-to-Showcase-The-Ambarella-Edge-From-Agentic-to-Physical-AI-at-Embedded-World-2026.html'}, {'name': 'Embedded World Official', 'url': 'https://www.embedded-world.de/'}, {'name': 'Ambarella Developer Zone Launch', 'url': 'https://www.ambarella.com/news/ambarella-launches-a-developer-zone-to-broaden-its-edge-ai-ecosystem/'}, {'name': 'Embedded Computing - CV7 at CES 2026', 'url': 'https://embeddedcomputing.com/technology/processing/chips-and-socs/ambarella-showcases-cv7-edge-ai-vision-soc-at-ces-2026'}, {'name': 'GlobeNewswire - CES 2026 Webcast', 'url': 'https://www.globenewswire.com/news-release/2026/01/02/3212080/23306/en/ambarella-announces-ces-2026-product-and-technology-briefing-webcast-on-january-6-2026.html'}], 'model_provider': 'claude', 'news_date': '2026-02-09', 'event_id': '314_2026-02-09'},
{'headline': 'AI Investors Ditch Software, Go All-In on Hardware', 'output': "The AI trade is splitting hard. Investors are rotating out of software stocks—spooked by AI disruption fears and $600B+ hyperscaler capex plans—and piling into hardware enablers. Samsung and SK Hynix are up 32% and 29% year-to-date on memory-chip demand. Meanwhile, U.S. names like ServiceNow and Salesforce tanked after Anthropic's Claude plug-ins threatened to automate workflows. Even the Magnificent 7 is fracturing as scrutiny over AI spending returns intensifies. This could reshape tech portfolios for years: picks-and-shovels over platforms.", 'sources': [{'name': 'The Journal Record', 'url': 'https://journalrecord.com/2026/02/09/global-ai-trade-fractures-investors-hardware-software/'}, {'name': 'Investing.com - Global software slide', 'url': 'https://www.investing.com/news/stock-market-news/global-software-data-firms-slide-as-ai-disruption-fears-compound-jitters-over-600-billion-capex-plans-4489820?utm_source=openai'}, {'name': 'Investing.com - Anthropic AI tools', 'url': 'https://www.investing.com/news/stock-market-news/anthropics-new-ai-tools-deepen-selloff-in-data-analytics-and-software-stocks-investors-say-4483600?utm_source=openai'}, {'name': 'Investing.com - US software tumble', 'url': 'https://www.investing.com/news/stock-market-news/us-software-stocks-tumble-sparks-concerns-that-ai-trade-is-reshaping-markets-4493058?utm_source=openai'}, {'name': 'The Guardian', 'url': 'https://www.theguardian.com/business/live/2026/feb/04/software-stock-selloff-ai-led-disruption-jensen-huang-services-economy-business-live-news-updates?page=with%3Ablock-6982f8b08f085ffa2d6a22c6'}], 'model_provider': 'claude', 'news_date': '2026-02-09', 'event_id': '315_2026-02-09'},
{'headline': 'Databricks Just Raised $7B and Is Now Worth $134B', 'output': "Databricks closed a monster funding round in February 2026—roughly $5B in equity plus ~$2B in debt capacity—cementing its $134B valuation. That's a wild jump from $62B just 13 months earlier (January 2025, when Meta came aboard). The cash will fuel Lakebase, Genie, and other AI tools as revenue hit a $5.4B run rate, up 65% year-over-year. For context: the data platform went from $100B (August 2025) to $134B in six months, powered by acquisitions like MosaicML ($1.3B, 2023) and Tabular (2024). This could cement Databricks as the enterprise AI infrastructure play ahead of a likely IPO.", 'sources': [{'name': 'PRNewswire (Feb 2026)', 'url': 'https://www.prnewswire.com/news-releases/databricks-grows-65-yoy-surpasses-5-4-billion-revenue-run-rate-doubles-down-on-lakebase-and-genie-302682674.html?utm_source=openai'}, {'name': 'Investing.com', 'url': 'https://www.investing.com/news/stock-market-news/databricks-valued-at-134-billion-in-latest-fundraise-cnbc-reports-4494367?utm_source=openai'}, {'name': 'TechCrunch (Jan 2025)', 'url': 'https://techcrunch.com/2025/01/22/databricks-closes-15-3b-financing-at-62b-valuation-meta-joins-as-strategic-investor/?utm_source=openai'}, {'name': 'CNBC', 'url': 'https://www.cnbc.com/2025/08/19/databricks-valuation-over-100-billion.html?utm_source=openai'}, {'name': 'PRNewswire (MosaicML)', 'url': 'https://www.prnewswire.com/news-releases/databricks-signs-definitive-agreement-to-acquire-mosaicml-a-leading-generative-ai-platform-301863046.html?utm_source=openai'}, {'name': 'TechCrunch (Tabular)', 'url': 'https://techcrunch.com/2024/06/04/databricks-acquires-tabular-to-build-a-common-data-lakehouse-standard/?utm_source=openai'}], 'model_provider': 'claude', 'news_date': '2026-02-09', 'event_id': '316_2026-02-09'},
{'headline': 'xAI Just Scored $5.4B in NVIDIA GPUs on Lease', 'output': "Elon Musk's xAI locked down a massive $5.4 billion deal for data center infrastructure packed with NVIDIA's latest GB200 GPUs—but here's the twist: it's leased, not bought. Apollo Global led $3.5B in financing through Valor Compute Infrastructure, with NVIDIA itself investing as an anchor partner. The triple-net lease structure gives investors quarterly payouts while powering one of the world's most powerful AI clusters for xAI's Grok system. This could signal a new financing model for cash-hungry AI startups needing cutting-edge compute without the upfront capital hit.", 'sources': [{'name': 'Apollo Global Management', 'url': 'https://ir.apollo.com/news-events/press-releases/detail/599/apollo-backs-5-4-billion-valor-and-xai-data-center-compute'}, {'name': 'Investing.com', 'url': 'https://www.investing.com/news/stock-market-news/apollo-global-management-nears-34-billion-loan-deal-for-nvidia-chips-to-xai-93CH-4494061'}], 'model_provider': 'claude', 'news_date': '2026-02-09', 'event_id': '317_2026-02-09'},
{'headline': 'Ex-Salesforce Co-CEO Bets $40M on Warehouse Drones', 'output': 'Gather AI just scored $40M in Series B funding led by Smith Point Capital—the firm of former Salesforce co-CEO Keith Block. The Pittsburgh startup, founded by Carnegie Mellon PhDs, uses off-the-shelf cameras and drones to monitor warehouse ops and feed data into management systems. With clients like Kwik Trip and Axon, plus a 2025 Nebius Vision AI award under their belt, Gather AI now has $74M total raised and a 60-person team. This could position them as the go-to AI infrastructure for logistics in an increasingly automated supply chain.', 'sources': [{'name': 'TechCrunch', 'url': 'https://techcrunch.com/2026/02/09/gather-ai-maker-of-curious-warehouse-drones-lands-40m-led-by-keith-blocks-firm/'}, {'name': 'Nebius', 'url': 'https://nebius.com/newsroom/nebius-debuts-the-robotics-and-physical-ai-awards-and-summit-to-support-next-generation-startups-with-1-5-million-in-compute-credits'}, {'name': 'Gather AI', 'url': 'https://www.gather.ai/press-releases'}], 'model_provider': 'claude', 'news_date': '2026-02-09', 'event_id': '318_2026-02-09'},
{'headline': 'Big Tech to Drop $650B on AI in 2026', 'output': "Google, Amazon, Meta, and Microsoft are set to blow roughly $650 billion in 2026 on AI infrastructure and data centers—a massive jump from 2025. Amazon leads at $200B (up from $132B), Google's spending $175-185B (from $91B), Meta's at $115-135B, and Microsoft's projected around $150B. Amazon's cash is also hitting chips, robotics, and satellites. This could cement these giants' AI dominance while smaller players struggle to keep pace with the infrastructure arms race.", 'sources': [{'name': 'Bloomberg', 'url': 'https://www.bloomberg.com/news/articles/2026-02-06/how-much-is-big-tech-spending-on-ai-computing-a-staggering-650-billion-in-2026?sref=r4AzvICB&srnd=homepage-americas&utm_source=openai'}, {'name': 'TechCrunch', 'url': 'https://techcrunch.com/2026/02/05/amazon-and-google-are-winning-the-ai-capex-race-but-whats-the-prize/'}], 'model_provider': 'claude', 'news_date': '2026-02-09', 'event_id': '319_2026-02-09'},
{'headline': 'Databricks Raises $7B, Now Worth More Than Disney', 'output': 'Databricks just secured over $7 billion in fresh capital ($5B equity, $2B debt) at a staggering $134 billion valuation, backed by JPMorgan Chase and other major players. The data analytics giant hit a $5.4 billion revenue run-rate with 65% year-over-year growth in Q4, plus positive free cash flow. The war chest will fuel two big bets: Lakebase, a serverless Postgres database for AI agents, and Genie, a conversational AI assistant for querying company data. With 800+ customers spending over $1M annually and AI revenue at $1.4B, Databricks is positioning itself as the infrastructure backbone for the AI agent era.', 'sources': [{'name': 'PR Newswire - Databricks Q4 2026 Announcement', 'url': 'https://www.prnewswire.com/news-releases/databricks-grows-65-yoy-surpasses-5-4-billion-revenue-run-rate-doubles-down-on-lakebase-and-genie-302682674.html?utm_source=openai'}, {'name': 'Yahoo Finance - Databricks Valuation', 'url': 'https://finance.yahoo.com/news/databricks-valued-134-billion-latest-151316758.html/?utm_source=openai'}], 'model_provider': 'claude', 'news_date': '2026-02-09', 'event_id': '320_2026-02-09'},
{'headline': 'Did Iambic Really Partner With Takeda? Not So Fast.', 'output': "Iambic Therapeutics' rumored multi-year collaboration with Takeda for AI-driven oncology and GI programs? Unverified. No public records from either company confirm it as of February 2026. What's real: Iambic's NeuralPLexer AI tech is legit, with proven partnerships including Revolution Medicines and Jazz Pharmaceuticals. The biotech startup even landed on CNBC's 2025 Disruptor 50 list. Takeda's been busy too—just not with Iambic. They've teamed up with Innovent and others. This could signal hype outpacing reality in AI-pharma announcements, so always double-check those press releases.", 'sources': [{'name': 'Takeda - Innovent Partnership', 'url': 'https://www.takeda.com/newsroom/newsreleases/2025/innovent/'}, {'name': 'Iambic - NeuralPLexer Research', 'url': 'https://www.businesswire.com/news/home/20240212651244/en/Iambic-Therapeutics-Announces-New-Research-Published-in-Nature-Machine-Intelligence-Demonstrating-the-Capabilities-of-Its-Generative-AI-NeuralPLexer-Technology-to-Predict-Protein-Ligand-Complex-Structures'}, {'name': 'Iambic - Revolution Medicines Collaboration', 'url': 'https://www.iambic.ai/post/revolution-medicines-collaboration'}, {'name': 'Iambic - Jazz Pharmaceuticals Collaboration', 'url': 'https://www.biospace.com/press-releases/iambic-announces-research-collaboration-with-jazz-pharmaceuticals-for-iam1363-and-zanidatamab-combination-cohort-in-her2-positive-breast-cancer'}, {'name': 'Iambic - CNBC Disruptor 50', 'url': 'https://www.businesswire.com/news/home/20250610916531/en/Iambic-Named-to-CNBCs-2025-Disruptor-50-List'}], 'model_provider': 'claude', 'news_date': '2026-02-09', 'event_id': '321_2026-02-09'},
{'headline': 'Popular AI Chat App Leaked 300M Messages. Oops.', 'output': 'Chat & Ask AI, a wrapper app connecting users to OpenAI, Anthropic, and Google models, exposed roughly 300 million messages from over 25 million users—half its claimed user base. Security researcher Harry discovered the mess: a Google Firebase misconfiguration left full chat histories wide open, including prompts about suicide and drug manufacturing. The app\'s developers patched it by January 20, but Firehound\'s registry showed 406,000+ exposed files lingering as of January 15. This could mean your "private" AI chats aren\'t so private after all.', 'sources': [{'name': '404 Media', 'url': 'https://www.404media.co/massive-ai-chat-app-leaked-millions-of-users-private-conversations/'}, {'name': 'Macworld', 'url': 'https://www.macworld.com/article/3037768/ai-apps-expose-your-data-on-the-app-store.html'}, {'name': 'Fox News', 'url': 'https://www.foxnews.com/tech/millions-ai-chat-messages-exposed-app-data-leak?utm_source=openai'}, {'name': 'AOL', 'url': 'https://www.aol.com/articles/millions-ai-chat-messages-exposed-131122138.html?utm_source=openai'}], 'model_provider': 'claude', 'news_date': '2026-02-09', 'event_id': '322_2026-02-09'},
{'headline': 'Lema AI Exits Stealth With $24M to Hunt Vendor Threats', 'output': 'Lema AI just dropped out of stealth with $24 million (Series A led by Team8, seed by F2 Venture Capital, plus Salesforce Ventures). The New York startup, founded in 2023, built an agentic AI platform that continuously monitors third-party vendors—think real-time security checks instead of boring compliance paperwork. It claims to assess vendor risk in under five minutes by mapping potential attack paths. Fortune 500s are already using it. With 60% of orgs juggling 1,000+ vendors and a third of breaches coming from third parties, this could reshape how enterprises handle supply chain security.', 'sources': [{'name': 'Business Insider', 'url': 'https://www.businessinsider.com/lema-has-come-out-of-stealth-to-raise-24-million-2026-2'}, {'name': 'SecurityWeek', 'url': 'https://www.securityweek.com/lema-ai-emerges-from-stealth-with-24-million-to-tackle-third-party-risk/amp/'}, {'name': 'Ynetnews', 'url': 'https://www.ynetnews.com/business/article/ryqatqpdze'}], 'model_provider': 'claude', 'news_date': '2026-02-09', 'event_id': '323_2026-02-09'},
{'headline': 'Cloudera Brings AI Inference On-Prem With NVIDIA Power', 'output': 'Cloudera just dropped AI inference and data warehouse capabilities directly into your data center. The February 2026 release brings Cloudera AI Inference (powered by NVIDIA Blackwell GPUs and NIM microservices) and Trino-based data warehousing on-premises, letting orgs run AI models behind their firewall. Enhanced Data Visualization adds AI annotation, query tracing, and governance features across cloud and edge environments. This could be huge for enterprises obsessed with data sovereignty and air-gapped deployments—finally, GenAI without the cloud dependency nightmare.', 'sources': [{'name': 'GlobeNewswire', 'url': 'https://www.globenewswire.com/news-release/2026/02/09/3234544/0/en/Cloudera-Unveils-Next-Phase-of-AI-Inferencing-and-Unified-Data-Access-Capabilities.html'}, {'name': 'Cloudera Press Release - November 2025', 'url': 'https://www.cloudera.com/about/news-and-blogs/press-releases/2025-11-20-cloudera-advances-unified-data-access-and-governance-with-ai-powered-federation-and-lineage.html'}, {'name': 'Cloudera AI Inference Service', 'url': 'https://www.cloudera.com/products/machine-learning/ai-inference-service.html'}], 'model_provider': 'claude', 'news_date': '2026-02-09', 'event_id': '324_2026-02-09'},
{'headline': 'Legal AI Unicorn Harvey Skyrockets to $11B in 12 Months', 'output': "Harvey, the legal AI startup founded in 2022, is reportedly raising $200M at an $11B valuation—led by Sequoia and Singapore's GIC. That's a wild jump from $3B (Feb 2025), to $5B (June), $8B (Dec), and now $11B in just one year. The company serves 50 of AmLaw 100 firms across 700 clients in 63 countries, with $190M ARR by end-2025. Four funding rounds in under 12 months signals either explosive traction or frothy AI hype. This could mean legal AI is finally proving its enterprise ROI—or VCs are betting big before reality checks in.", 'sources': [{'name': 'TechCrunch', 'url': 'https://techcrunch.com/2026/02/09/harvey-reportedly-raising-at-11b-valuation-just-months-after-it-hit-8b/'}, {'name': 'Fortune', 'url': 'https://fortune.com/2025/02/12/legal-ai-startup-harvey-300-million-series-d-funding-3-billion-valuation-sequoia/'}, {'name': 'TechCrunch', 'url': 'https://techcrunch.com/2025/06/23/four-months-after-a-3b-valuation-harvey-ai-grows-to-5b/'}, {'name': 'Bloomberg', 'url': 'https://www.bloomberg.com/news/articles/2025-10-29/andreessen-horowitz-invests-in-legal-ai-startup-harvey-at-an-8-billion-valuation'}, {'name': 'TechCrunch', 'url': 'https://techcrunch.com/2025/12/04/legal-ai-startup-harvey-confirms-8b-valuation/'}], 'model_provider': 'claude', 'news_date': '2026-02-09', 'event_id': '325_2026-02-09'},
{'headline': 'This AI Girlfriend Actually Remembers Your Conversations', 'output': "Dream Companion just launched an AI companion platform that won't ghost your emotional baggage. Unveiled February 9 in New York, the platform uses proprietary memory architecture to retain conversation history, personality traits, and relationship dynamics across sessions. Users can customize their AI's personality, tone, and backstory—yes, it supports NSFW modes with privacy controls. Conversations are encrypted, anonymized, and never shared with third parties. Currently web-based with mobile coming soon, sign-up is free. This could reshape digital intimacy as AI companions evolve from chatbots to persistent relationships with actual context retention.", 'sources': [{'name': 'GlobeNewswire', 'url': 'https://www.globenewswire.com/news-release/2026/02/09/3234840/0/en/Dream-Companion-Launches-Advanced-AI-Companion-Platform-Featuring-Long-Term-Memory-and-Personalized-Interaction.html'}], 'model_provider': 'claude', 'news_date': '2026-02-10', 'event_id': '326_2026-02-10'}]